In [1]:
import torch
import torchvision
import datetime
from torchvision import transforms
from torchvision.models import resnet18
from PIL import Image
import time
import pytz
# from jtop import jtop
# jetson = jtop()
# jetson.start()
# print("Before loading model: ",jetson.stats)
def read_classes():
    """
    Load the ImageNet class names.
    """
    with open("imagenet-classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    return categories

print("Before loading model: ",datetime.datetime.now(pytz.timezone('Europe/London')))
# Load the pre-trained ResNet model
model = torchvision.models.resnet18(pretrained=True)
# print(model)
categories = read_classes()
model.eval()  # Set the model to evaluation mode



Before loading model:  2024-10-18 17:02:37.795498+01:00


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
# Load and preprocess an example image
print("Loading image: ",datetime.datetime.now(pytz.timezone('Europe/London')))
image_path = 'cat_3.jpg'
input_image = Image.open(image_path).convert('RGB')
# print("Image transformation: ",jetson.stats)

Loading image:  2024-10-18 17:03:11.864517+01:00


In [3]:
print("Image transformation: ",datetime.datetime.now(pytz.timezone('Europe/London')))
# Define the image transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformations to the input image
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension


Image transformation:  2024-10-18 17:03:14.790826+01:00


In [4]:
# Make the prediction
with torch.no_grad():
    start_time = time.time()
    print("During prediction: ", datetime.datetime.now(pytz.timezone('Europe/London')))
    # print("During prediction: ",jtop().stats)
    output = model(input_batch)
    end_time = time.time()
# get the softmax probabilities
probabilities = torch.nn.functional.softmax(output[0], dim=0)
# check the top 5 categories that are predicted
top5_prob, top5_catid = torch.topk(probabilities, 1)
# Get the predicted class index
_, predicted_idx = torch.max(output, 1)
print(predicted_idx)
predicted_class = "Cats" if predicted_idx.item() == 0 else "Dogs"

# Print the predicted class label and inference time
inference_time = end_time - start_time
print(f'Predicted class: {predicted_class}')
print(f'Inference time: {inference_time:.4f} seconds')
print(f"Accuracy: {top5_prob[0].item()*100:.3f}%")
print(f"class:{categories[top5_catid[0]]}")      

During prediction:  2024-10-18 17:03:18.055401+01:00
tensor([287])
Predicted class: Dogs
Inference time: 0.2995 seconds
Accuracy: 99.623%
class:lynx, catamount


In [14]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total trainable parameters: {total_params}')

Total trainable parameters: 11689512


In [15]:
import torch
import torchvision.models as models

# Load the pre-trained Mobilenetv3 model
model = torchvision.models.resnet18(pretrained=True)

# Define input size (224x224x3) for Mobilenetv3
input_size = (224, 224, 3)

# Function to calculate FLOPs for the model
def count_flops(model, input_size):
    # Define a tensor of appropriate size
    input_tensor = torch.randn(1, *input_size)
    
    # Switch to evaluation mode
    model.eval()
    
    # Move model to appropriate device
    device = next(model.parameters()).device
    input_tensor = input_tensor.to(device)
    model.to(device)
    
    # Iterate through model's layers
    flops = 0
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            # For convolutional layers
            output_size = (input_size[0] - module.kernel_size[0] + 2 * module.padding[0]) // module.stride[0] + 1
            output_size = (output_size - module.kernel_size[1] + 2 * module.padding[1]) // module.stride[1] + 1
            flops += module.in_channels * module.out_channels * module.kernel_size[0] * module.kernel_size[1] * output_size * output_size
            input_size = (output_size, output_size, module.out_channels)
        elif isinstance(module, torch.nn.MaxPool2d):
            # For max pooling layers
            output_size = (input_size[0] - module.kernel_size) // module.stride + 1
            flops += input_size[2] * output_size * output_size
            input_size = (output_size, output_size, input_size[2])
        elif isinstance(module, torch.nn.Linear):
            # For fully connected layers
            flops += module.in_features * module.out_features
            input_size = (module.out_features,)
    
    return flops

# Calculate FLOPs
total_flops = count_flops(model, input_size)
print("Total FLOPs:", total_flops)

Total FLOPs: 160093760


Total FLOPs: 160093760

0: 1.jpg

datetime.datetime(2024, 10, 18, 15, 33, 54, 868198)
datetime.datetime(2024, 10, 18, 15, 33, 55, 868411)
datetime.datetime(2024, 10, 18, 15, 33, 56, 870046)
datetime.datetime(2024, 10, 18, 15, 33, 57, 870117)
datetime.datetime(2024, 10, 18, 15, 33, 58, 869754)
datetime.datetime(2024, 10, 18, 15, 33, 59, 873258)
datetime.datetime(2024, 10, 18, 15, 34, 00, 873588)
datetime.datetime(2024, 10, 18, 15, 34, 01, 871336)
datetime.datetime(2024, 10, 18, 15, 34, 02, 889960)
datetime.datetime(2024, 10, 18, 15, 34, 03, 887258)
datetime.datetime(2024, 10, 18, 15, 34, 04, 887650)
datetime.datetime(2024, 10, 18, 15, 34, 05, 882106)
datetime.datetime(2024, 10, 18, 15, 34, 06, 893992)
datetime.datetime(2024, 10, 18, 15, 34, 07, 884556)
datetime.datetime(2024, 10, 18, 15, 34, 08, 903638)
datetime.datetime(2024, 10, 18, 15, 34, 09, 928023)
datetime.datetime(2024, 10, 18, 15, 34, 10, 909856)
Average CPU Temperature: 28.47°C
Average GPU Temperature: 29.74°C
Average Power CPU: 818.88mW
Average Power GPU: 818.88mW
Average Power TOT: 2348.94mW
Average RAM: 0.2104

{'Avg_CPU_temp': 28.470588235294116, 'Avg_GPU_temp': 29.735294117647058, 'Avg_Power_CPU': 818.8823529411765, 'Avg_Power_GPU': 818.8823529411765, 'Avg_CPU_RAM': 0.2104423488140637}

1: cat_0
Loading image:  2024-10-18 16:46:20.909065+01:00
Image transformation:  2024-10-18 16:46:22.104554+01:00
During prediction:  2024-10-18 16:46:24.228559+01:00
tensor([281])
Predicted class: Dogs
Inference time: 0.3439 seconds
Accuracy: 18.534%
class:tabby, tabby cat

datetime.datetime(2024, 10, 18, 16, 48, 00, 130376)
datetime.datetime(2024, 10, 18, 16, 48, 01, 128240)
datetime.datetime(2024, 10, 18, 16, 48, 02, 131405)
datetime.datetime(2024, 10, 18, 16, 48, 03, 121917)
datetime.datetime(2024, 10, 18, 16, 48, 04, 148955)
datetime.datetime(2024, 10, 18, 16, 48, 05, 122257)
datetime.datetime(2024, 10, 18, 16, 48, 06, 135844)
datetime.datetime(2024, 10, 18, 16, 48, 07, 136782)
datetime.datetime(2024, 10, 18, 16, 48, 08, 134811)
datetime.datetime(2024, 10, 18, 16, 48, 09, 128557)
datetime.datetime(2024, 10, 18, 16, 48, 10, 141018)
datetime.datetime(2024, 10, 18, 16, 48, 11, 146300)
datetime.datetime(2024, 10, 18, 16, 48, 12, 139647)
datetime.datetime(2024, 10, 18, 16, 48, 13, 132430)
datetime.datetime(2024, 10, 18, 16, 48, 14, 143442)

Average CPU Temperature: 30.40°C
Average GPU Temperature: 31.70°C
Average Power CPU: 652.20mW
Average Power GPU: 652.20mW
Average Power TOT: 2098.60mW
Average RAM: 0.4730

{'Avg_CPU_temp': 30.4, 'Avg_GPU_temp': 31.7, 'Avg_Power_CPU': 652.2, 'Avg_Power_GPU': 652.2, 'Avg_CPU_RAM': 0.47296065273302396}

2: cat_1
Loading image:  2024-10-18 16:48:01.532727+01:00
Image transformation:  2024-10-18 16:48:04.110424+01:00
During prediction:  2024-10-18 16:48:12.566883+01:00
tensor([281])
Predicted class: Dogs
Inference time: 0.2974 seconds
Accuracy: 64.716%
class:tabby, tabby cat

datetime.datetime(2024, 10, 18, 16, 48, 00, 130376)
datetime.datetime(2024, 10, 18, 16, 48, 01, 128240)
datetime.datetime(2024, 10, 18, 16, 48, 02, 131405)
datetime.datetime(2024, 10, 18, 16, 48, 03, 121917)
datetime.datetime(2024, 10, 18, 16, 48, 04, 148955)
datetime.datetime(2024, 10, 18, 16, 48, 05, 122257)
datetime.datetime(2024, 10, 18, 16, 48, 06, 135844)
datetime.datetime(2024, 10, 18, 16, 48, 07, 136782)
datetime.datetime(2024, 10, 18, 16, 48, 08, 134811)
datetime.datetime(2024, 10, 18, 16, 48, 09, 128557)
datetime.datetime(2024, 10, 18, 16, 48, 10, 141018)
datetime.datetime(2024, 10, 18, 16, 48, 11, 146300)
datetime.datetime(2024, 10, 18, 16, 48, 12, 139647)
datetime.datetime(2024, 10, 18, 16, 48, 13, 132430)

Average CPU Temperature: 30.61°C
Average GPU Temperature: 31.89°C
Average Power CPU: 674.57mW
Average Power GPU: 674.57mW
Average Power TOT: 2074.79mW
Average RAM: 0.4728

{'Avg_CPU_temp': 30.607142857142858, 'Avg_GPU_temp': 31.892857142857142, 'Avg_Power_CPU': 674.5714285714286, 'Avg_Power_GPU': 674.5714285714286, 'Avg_CPU_RAM': 0.47276098129558386}

3: dog_0 
Loading image:  2024-10-18 16:54:27.377465+01:00
Image transformation:  2024-10-18 16:54:31.292117+01:00
During prediction:  2024-10-18 16:54:37.215338+01:00
tensor([249])
Predicted class: Dogs
Inference time: 0.3032 seconds
Accuracy: 89.873%
class:malamute, malemute, Alaskan malamute

datetime.datetime(2024, 10, 18, 16, 54, 27, 672386)
datetime.datetime(2024, 10, 18, 16, 54, 28, 669776)
datetime.datetime(2024, 10, 18, 16, 54, 29, 676345)
datetime.datetime(2024, 10, 18, 16, 54, 30, 666303)
datetime.datetime(2024, 10, 18, 16, 54, 31, 674525)
datetime.datetime(2024, 10, 18, 16, 54, 32, 679884)
datetime.datetime(2024, 10, 18, 16, 54, 33, 678402)
datetime.datetime(2024, 10, 18, 16, 54, 34, 671880)
datetime.datetime(2024, 10, 18, 16, 54, 35, 685031)
datetime.datetime(2024, 10, 18, 16, 54, 36, 702716)
datetime.datetime(2024, 10, 18, 16, 54, 37, 680425)

Average CPU Temperature: 30.68°C
Average GPU Temperature: 31.55°C
Average Power CPU: 683.82mW
Average Power GPU: 683.82mW
Average Power TOT: 2043.55mW
Average RAM: 0.4729

{'Avg_CPU_temp': 30.681818181818183, 'Avg_GPU_temp': 31.545454545454547, 'Avg_Power_CPU': 683.8181818181819, 'Avg_Power_GPU': 683.8181818181819, 'Avg_CPU_RAM': 0.4729157540462119}

4: cat_2 
Before loading model:  2024-10-18 16:33:03.567822+01:00
Image transformation:  2024-10-18 16:40:36.751187+01:00
During prediction:  2024-10-18 16:40:39.892731+01:00
tensor([283])
Predicted class: Dogs
Inference time: 0.3723 seconds
Accuracy: 99.256%
class:Persian cat

datetime.datetime(2024, 10, 18, 16, 40, 28, 481964)
datetime.datetime(2024, 10, 18, 16, 40, 29, 488857)
datetime.datetime(2024, 10, 18, 16, 40, 30, 487460)
datetime.datetime(2024, 10, 18, 16, 40, 31, 489836)
datetime.datetime(2024, 10, 18, 16, 40, 32, 486765)
datetime.datetime(2024, 10, 18, 16, 40, 33, 494905)
datetime.datetime(2024, 10, 18, 16, 40, 34, 495803)
datetime.datetime(2024, 10, 18, 16, 40, 35, 499637)
datetime.datetime(2024, 10, 18, 16, 40, 36, 502526)
datetime.datetime(2024, 10, 18, 16, 40, 37, 495552)
datetime.datetime(2024, 10, 18, 16, 40, 38, 498921)
datetime.datetime(2024, 10, 18, 16, 40, 39, 501707)
datetime.datetime(2024, 10, 18, 16, 40, 40, 493395)

Average CPU Temperature: 30.58°C
Average GPU Temperature: 31.62°C
Average Power CPU: 716.62mW
Average Power GPU: 716.62mW
Average Power TOT: 2114.38mW
Average RAM: 0.4728

{'Avg_CPU_temp': 30.576923076923077, 'Avg_GPU_temp': 31.615384615384617, 'Avg_Power_CPU': 716.6153846153846, 'Avg_Power_GPU': 716.6153846153846, 'Avg_CPU_RAM': 0.47278330994888773}

5: dog_1
Loading image:  2024-10-18 16:56:22.115990+01:00
Image transformation:  2024-10-18 16:56:26.635307+01:00
During prediction:  2024-10-18 16:56:30.512122+01:00
tensor([256])
Predicted class: Dogs
Inference time: 0.4154 seconds
Accuracy: 63.549%
class:Newfoundland, Newfoundland dog

datetime.datetime(2024, 10, 18, 16, 56, 22, 827212)
datetime.datetime(2024, 10, 18, 16, 56, 23, 837316)
datetime.datetime(2024, 10, 18, 16, 56, 24, 827328)
datetime.datetime(2024, 10, 18, 16, 56, 25, 847038)
datetime.datetime(2024, 10, 18, 16, 56, 26, 835788)
datetime.datetime(2024, 10, 18, 16, 56, 27, 840432)
datetime.datetime(2024, 10, 18, 16, 56, 28, 831363)
datetime.datetime(2024, 10, 18, 16, 56, 29, 841117)
datetime.datetime(2024, 10, 18, 16, 56, 30, 896692)
datetime.datetime(2024, 10, 18, 16, 56, 31, 842305)
Average CPU Temperature: 30.65°C
Average GPU Temperature: 31.65°C
Average Power CPU: 758.60mW
Average Power GPU: 758.60mW
Average Power TOT: 2159.50mW
Average RAM: 0.4729

{'Avg_CPU_temp': 30.65, 'Avg_GPU_temp': 31.65, 'Avg_Power_CPU': 758.6, 'Avg_Power_GPU': 758.6, 'Avg_CPU_RAM': 0.4729410431509347}